# Module 3: Particle transport by mean velocity

We already defined and populated `U` and `V` fields which represent mean velocities averaged over 10 years. <br>
In this module we will use these velocity fields to transport particles across the globe

Let's include everything we have done in the previous modules (everything is contained in `module_02.jl`)
and check that our building blocks are there (grid, U and V)

In [1]:
using NBInclude 

@nbinclude("02_load_and_visualize_velocity_data.ipynb")

@show grid;
@show U;
@show V;

    Updating registry at `/srv/julia/pkg/registries/General.toml`
   Resolving package versions...
  No Changes to `/srv/julia/pkg/environments/v1.7/Project.toml`
  No Changes to `/srv/julia/pkg/environments/v1.7/Manifest.toml`


size(bathymetry) = (1440, 600)
typeof(bathymetry) = Matrix{Float64}
grid = 360×180×1 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo:
├── immersed_boundary: GridFittedBoundary{Field{Center, Center, Center, Nothing, LatitudeLongitudeGrid{Float64, Periodic, Bounded, Bounded, OffsetArrays.OffsetVector{Float64, Vector{Float64}}, Float64, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, CPU}, Tuple{Colon, Colon, Colon}, OffsetArrays.OffsetArray{Bool, 3, Array{Bool, 3}}, Bool, FieldBoundaryConditions{BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.BoundaryCo

LoadError: LoadError: UndefVarError: mask not defined
in expression starting at /home/jovyan/coessing2022-hub/julia-ocean-model/02_load_and_visualize_velocity_data.ipynb:In[19]:8

### Defining particle initial positions

particles in Oceananigans are defined by an x, y and z position. (In case of a spherical domain x and y are longitude and latitude, respectively)

Initial positions have to be defined as arrays, let's distribute the initial positions around a <br>
center with some random values:

$$\lambda_p = \lambda_c + S * (R - 0.5)$$

where $R$ is a random value between 0 and 1 and $S$ controls the spread of the initial positions

In [28]:
λ₀, φ₀ = -46.0, 3.5
λ₀, φ₀ = -75.0, 30.5
λ₀, φ₀ = -120.0, -60.0

degree_spread_λ = 2.0
degree_spread_φ = 2.0

n_particles = 100

λₚ = λ₀ .+ degree_spread_λ .* (rand(n_particles) .- 0.5);
φₚ = φ₀ .+ degree_spread_φ .* (rand(n_particles) .- 0.5);

zₚ = 0.5 .* ones(n_particles);

@show extrema(λₚ);
@show extrema(φₚ);

extrema(λₚ) = (-120.98228949647124, -119.04236302511484)
extrema(φₚ) = (-60.9908929347577, -59.012127149835464)


### Particle in Oceananigans

we just feed the arrays containing the initial positions to the particles <br>
constructor and this will create a `LagrangianParticle` object

In [29]:
lagrangian_particles = LagrangianParticles(x=λₚ, y=φₚ, z=zₚ)

100 LagrangianParticles with eltype Particle:
├── 3 properties: (:x, :y, :z)
├── particle-wall restitution coefficient: 1.0
├── 0 tracked fields: ()
└── dynamics: no_dynamics

### Last building blocks: Model and Simulation

A model in Oceananigans includes all the physical phenomena we want to represent. <br>

In our case we only want to have particles transported by the velocity fields we previously defined. <br>



In [30]:

model = HydrostaticFreeSurfaceModel(grid = grid, velocities = PrescribedVelocityFields(u = U, v = V),
                                    buoyancy  = nothing,
                                    tracers   = (),
                                    particles = lagrangian_particles)

HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── grid: 360×180×1 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo
├── timestepper: QuasiAdamsBashforth2TimeStepper
├── tracers: ()
├── closure: Nothing
├── buoyancy: Nothing
├── coriolis: Nothing
└── particles: 100 Lagrangian particles with 3 properties: (:x, :y, :z)

### Defining a simulation

The last step before running a model is to define a Simulation. <br>
A Simulation in Oceananigans is a wrapper around the model which specifies <br>
all the additional technical details of the simulation:
- What is the time step? 
- When are we stopping the simulation?

In [31]:
Δt = 6hours

simulation = Simulation(model, Δt = Δt, stop_time = 10years);

start_time = [time_ns()]

simulation.callbacks[:progress] = Callback(progress, IterationInterval(500));
save_interval = 10days;

simulation.output_writers[:surface_fields] = JLD2OutputWriter(model,  (; u = model.velocities.u, v = model.velocities.v, particles=model.particles,),
                            schedule = TimeInterval(save_interval),
                            filename = "output_particles",
                            overwrite_existing = true);
@show simulation;

simulation = Simulation of HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 6 hours
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN years
├── Stop time: 10 years
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 5 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   ├── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
│   └── progress => Callback of progress on IterationInterval(500)
├── Output writers: OrderedDict with 1 entry:
│   └── surface_fields => JLD2OutputWriter writing (u, v, particles) to ./output_particles.jld2 on TimeInterval(10 days)
└── Diagnostics: OrderedDict with no entries


In [32]:
Δt = 6hours

simulation = Simulation(model, Δt = Δt, stop_time = 10years);

start_time = [time_ns()]

simulation.callbacks[:progress] = Callback(progress, IterationInterval(500));
save_interval = 10days;

simulation.output_writers[:surface_fields] = JLD2OutputWriter(model,  (; u = model.velocities.u, v = model.velocities.v, particles=model.particles,),
                            schedule = TimeInterval(save_interval),
                            filename = "output_particles",
                            overwrite_existing = true);
@show simulation;

simulation = Simulation of HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 6 hours
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN years
├── Stop time: 10 years
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 5 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   ├── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
│   └── progress => Callback of progress on IterationInterval(500)
├── Output writers: OrderedDict with 1 entry:
│   └── surface_fields => JLD2OutputWriter writing (u, v, particles) to ./output_particles.jld2 on TimeInterval(10 days)
└── Diagnostics: OrderedDict with no entries


### Let's run our first simulation!

Every ingredient is complete, to run the simulation we can just:

In [33]:
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /srv/julia/pkg/packages/Oceananigans/HYqGf/src/Simulations/run.jl:167
┌ Info: Time:    0 seconds, iteration: 0, wall time: 1.160 seconds
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/useful_functions.jl:96
┌ Info:     ... simulation initialization complete (50.697 ms)
└ @ Oceananigans.Simulations /srv/julia/pkg/packages/Oceananigans/HYqGf/src/Simulations/run.jl:202
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /srv/julia/pkg/packages/Oceananigans/HYqGf/src/Simulations/run.jl:112
┌ Info:     ... initial time step complete (237.733 μs).
└ @ Oceananigans.Simulations /srv/julia/pkg/packages/Oceananigans/HYqGf/src/Simulations/run.jl:119
┌ Info: Time:     125 days, iteration: 500, wall time: 533.792 ms
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/useful_functions.jl:96
┌ Info: Time:     250 days, iteration: 1000, wall time: 429.785 ms
└ @ Main /home/jovyan/coessing2022-hub

### Visualize the output

I have already defined a visualization function for the output in "visualize_particles.jl". <br>
(If you want to take a look, go ahead and ask if there are questions)

In [34]:
include("utils/visualize_particles.jl")
visualize_results("output_particles")

┌ Info: Plotting iteration 50 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 100 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 150 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 200 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 250 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 300 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 350 of 366...
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: filename = output_particles.mp4
└ @ Main /home/jovyan/coessing2022-hub/julia-ocean-

In [35]:
display_mp4("output_particles.mp4")

# Exericse: Try it yourself!

Try changing the particle parameters and see where the particles end up
- initial position 
- spread
- number of particles

You should see particles accumulate in "convergence" regions  <br>
Some interesting positions to try are:

***Southern Ocean*** <br>
λ₀, φ₀ = -120.0, -60.0

***Equatorial Pacific Ocean*** <br>
λ₀, φ₀ = -160.0, 0.0

***West Africa Coast*** <br>
λ₀, φ₀ = 55.2, 8.3

***Gulf stream (north Atlantic)*** <br>
λ₀, φ₀ = -75.0, 30.5